In [ ]:
from PIL import Image
from os import listdir
from os.path import isdir
from numpy import asarray, expand_dims
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def load_face(filename):
    #carregando imagem de arquivo
    image = Image.open(filename)
    
    #converter para RGB
    image = image.convert("RGB")
    
    return asarray(image)

In [ ]:
def load_faces(directory_src):
    
    faces = list()
    
    # iterando arquivos
    for filename in listdir(directory_src):
        
        path = directory_src + filename
        
        try:
            faces.append(load_face(path))
        except:
            print("Erro na imagem {}".format(path))
    
    return faces

In [ ]:
def load_fotos(directory_src):
    
    X,y = list(), list()
    
    # iterar pastas por classes 
    for subdir in listdir(directory_src):
        
        #path
        path = directory_src + subdir + "/"
        
        if not isdir(path):
            continue
            
        faces = load_faces(path)
        
        labels = [subdir for _ in range(len(faces))]
        
        # sumarizar progresso
        print(' >Carregadas %d faces da classe: %s' % (len(faces), subdir))
        
        X.extend(faces)
        y.extend(labels)
        
    return asarray(X),asarray(y)

In [ ]:
trainX, trainy = load_fotos(directory_src = "/content/drive/My Drive/Colab Notebooks/faces/")

 >Carregadas 148 faces da classe: Nayara
 >Carregadas 150 faces da classe: Guilherme


In [ ]:
trainX.shape

(298, 160, 160, 3)

In [ ]:
trainy.shape

(298,)

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model = load_model('/content/drive/My Drive/Colab Notebooks/facenet_keras.h5')

In [ ]:
model.summary()

In [ ]:
def get_embedding(model, face_pixels):

  #padronização
  mean, std = face_pixels.mean(), face_pixels.std()
  face_pixels = (face_pixels - mean)/std

  #Transformar a face em 1 único exemplo 

  ## (160, 160) -> (1, 160, 160)

  samples = expand_dims(face_pixels, axis=0)

  #Realizar a predição gerando o embedding
  yhat = model.predict(samples)

  ## [[1,2..128], [1,2.. 128]]

  return yhat[0]

In [ ]:
newTrainX = list()

In [ ]:
for face in trainX:
  embedding = get_embedding(model ,face)
  newTrainX.append(embedding)

In [ ]:
newTrainX = asarray(newTrainX)

In [ ]:
newTrainX.shape

(298, 128)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(data= newTrainX)

In [ ]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127
0,0.584225,0.804500,0.452854,-1.623824,-2.100882,-0.574902,1.179415,-0.885701,-1.153842,-0.518011,0.867068,1.357912,0.909454,-0.495205,1.326498,2.195339,-0.460196,-0.471430,-0.576819,1.115017,-0.685666,-0.798408,0.072348,-0.304713,0.615707,-1.472816,0.886068,0.665374,-0.267587,0.995600,-1.349110,-0.893656,0.545359,0.248728,-0.086062,-0.875060,-0.454620,-1.330075,-0.863409,0.540738,...,0.178935,-0.393734,0.139024,-0.276476,1.418241,-0.521715,-1.046467,1.827067,-0.106728,-0.323344,-1.684466,-0.104599,-0.046590,-0.111518,1.458079,1.816486,-0.693711,0.114979,-0.353302,0.255499,-0.787746,0.836923,0.334541,0.676725,1.783558,-0.786473,0.105471,-0.303442,0.394270,-0.071311,-0.634817,0.449954,1.168273,-0.379423,0.650725,-1.596077,-0.639775,2.117943,0.523293,-0.808841
1,0.199379,-0.417365,-0.013070,-0.868356,-1.429072,-0.416843,0.771308,-1.052644,-0.874901,0.925591,0.628215,1.080636,0.457416,0.245152,0.880251,1.806331,-0.328360,-0.065741,0.965473,2.023221,-0.420611,-0.204448,-0.202459,-0.445019,0.258997,-0.479691,-0.189432,-0.822731,1.693441,0.410132,-0.849425,-0.752824,1.367362,-0.383551,-0.206330,0.764584,-0.194948,-0.983297,0.088132,-1.743614,...,0.278154,-0.080916,0.099688,0.966433,1.410327,1.417300,-2.131412,0.084986,0.901203,0.133180,-1.146312,-0.443624,-0.290889,-0.005500,0.367555,1.179283,-0.881916,0.484248,0.843655,-0.273589,-0.913899,0.828757,0.672520,1.038065,1.266932,-1.963843,1.265124,-0.751476,1.411388,-0.960451,-0.231233,0.693235,0.661702,0.655226,0.736951,-1.609178,-0.638518,1.880501,0.121869,0.266469
2,0.452089,-0.360339,-0.331302,-0.837509,-1.534547,-0.825833,-1.079196,-0.670622,-2.338496,1.605420,-0.932580,1.138811,1.492900,-0.177298,2.321504,1.395335,-0.869406,-0.338416,-0.003428,0.619495,0.830586,-0.524107,0.882610,-0.181550,-0.584979,-1.894311,0.544872,1.605167,-0.236215,0.345130,-1.658911,-0.408596,0.791158,0.020560,0.177980,-1.282815,0.625530,-1.100141,0.494570,-0.223549,...,-0.080115,-1.628601,-0.008485,0.177018,1.178912,1.144787,0.650706,-0.289966,0.683916,-0.727858,-0.542786,-0.805278,-0.244894,0.417187,0.084574,1.124941,-1.279079,-0.985738,0.842657,-0.490325,0.047757,1.318692,0.484380,-0.212093,0.655251,-0.421447,0.079555,0.271234,1.585232,0.648166,-1.341601,0.628314,-0.636974,1.040308,0.503656,-1.441806,-1.404229,2.494195,0.614774,-0.728288
3,-0.430537,0.127860,-0.213254,0.906390,-0.852794,-1.365903,-0.161194,-0.753833,-0.799612,1.446837,-0.272071,1.567998,1.207384,-0.338296,1.608089,1.150430,-0.634676,-0.254125,-1.071040,1.211796,-0.278586,-0.525854,0.748559,0.164741,-0.481178,-1.675388,0.875207,0.975484,-0.257127,-0.012251,-2.161046,-0.796146,-0.836062,-0.694677,0.361975,-0.950141,-0.702509,-0.983462,0.284671,-1.131678,...,-1.017007,-1.792642,0.489132,0.707611,0.945068,0.635493,0.032225,-0.331405,1.627035,1.052389,-1.286518,-0.632650,-1.188038,-0.012820,0.857500,1.356536,-0.997797,0.729610,1.785720,-0.313547,-0.603171,1.687713,0.873078,0.883546,0.837203,-0.717129,-0.011016,0.617468,1.562592,0.885327,-1.959883,0.699144,0.089146,0.059228,0.732675,-1.229492,-1.434176,1.968918,0.379001,-0.353593
4,-0.107481,-0.689445,-0.480524,0.131852,-1.843418,-0.476712,-0.859758,-1.044694,-1.539799,1.486555,-0.880955,1.304679,1.061523,0.134768,2.604447,0.981011,-0.818154,-0.131803,-0.571664,1.291709,-0.072231,-1.144796,1.688066,0.264710,0.972808,-2.386386,1.039902,1.534913,0.114218,0.220803,-0.803907,-1.023913,1.121686,0.105092,0.038132,-1.248504,1.253716,-1.648165,-0.489833,-0.070276,...,1.333755,-1.092080,-0.621510,-1.314188,0.868450,0.994173,-0.501640,0.800490,0.194400,-0.756526,-1.231052,-0.405903,-0.023578,0.909067,0.289566,1.403794,-0.446976,-1.125021,0.379413,-0.716104,1.348528,0.722898,0.136608,-0.175436,1.756393,-1.363832,-0.106016,-0.615542,0.843601,0.6340

In [ ]:
df['target'] = trainy

In [ ]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,target
0,0.584225,0.804500,0.452854,-1.623824,-2.100882,-0.574902,1.179415,-0.885701,-1.153842,-0.518011,0.867068,1.357912,0.909454,-0.495205,1.326498,2.195339,-0.460196,-0.471430,-0.576819,1.115017,-0.685666,-0.798408,0.072348,-0.304713,0.615707,-1.472816,0.886068,0.665374,-0.267587,0.995600,-1.349110,-0.893656,0.545359,0.248728,-0.086062,-0.875060,-0.454620,-1.330075,-0.863409,0.540738,...,-0.393734,0.139024,-0.276476,1.418241,-0.521715,-1.046467,1.827067,-0.106728,-0.323344,-1.684466,-0.104599,-0.046590,-0.111518,1.458079,1.816486,-0.693711,0.114979,-0.353302,0.255499,-0.787746,0.836923,0.334541,0.676725,1.783558,-0.786473,0.105471,-0.303442,0.394270,-0.071311,-0.634817,0.449954,1.168273,-0.379423,0.650725,-1.596077,-0.639775,2.117943,0.523293,-0.808841,Nayara
1,0.199379,-0.417365,-0.013070,-0.868356,-1.429072,-0.416843,0.771308,-1.052644,-0.874901,0.925591,0.628215,1.080636,0.457416,0.245152,0.880251,1.806331,-0.328360,-0.065741,0.965473,2.023221,-0.420611,-0.204448,-0.202459,-0.445019,0.258997,-0.479691,-0.189432,-0.822731,1.693441,0.410132,-0.849425,-0.752824,1.367362,-0.383551,-0.206330,0.764584,-0.194948,-0.983297,0.088132,-1.743614,...,-0.080916,0.099688,0.966433,1.410327,1.417300,-2.131412,0.084986,0.901203,0.133180,-1.146312,-0.443624,-0.290889,-0.005500,0.367555,1.179283,-0.881916,0.484248,0.843655,-0.273589,-0.913899,0.828757,0.672520,1.038065,1.266932,-1.963843,1.265124,-0.751476,1.411388,-0.960451,-0.231233,0.693235,0.661702,0.655226,0.736951,-1.609178,-0.638518,1.880501,0.121869,0.266469,Nayara
2,0.452089,-0.360339,-0.331302,-0.837509,-1.534547,-0.825833,-1.079196,-0.670622,-2.338496,1.605420,-0.932580,1.138811,1.492900,-0.177298,2.321504,1.395335,-0.869406,-0.338416,-0.003428,0.619495,0.830586,-0.524107,0.882610,-0.181550,-0.584979,-1.894311,0.544872,1.605167,-0.236215,0.345130,-1.658911,-0.408596,0.791158,0.020560,0.177980,-1.282815,0.625530,-1.100141,0.494570,-0.223549,...,-1.628601,-0.008485,0.177018,1.178912,1.144787,0.650706,-0.289966,0.683916,-0.727858,-0.542786,-0.805278,-0.244894,0.417187,0.084574,1.124941,-1.279079,-0.985738,0.842657,-0.490325,0.047757,1.318692,0.484380,-0.212093,0.655251,-0.421447,0.079555,0.271234,1.585232,0.648166,-1.341601,0.628314,-0.636974,1.040308,0.503656,-1.441806,-1.404229,2.494195,0.614774,-0.728288,Nayara
3,-0.430537,0.127860,-0.213254,0.906390,-0.852794,-1.365903,-0.161194,-0.753833,-0.799612,1.446837,-0.272071,1.567998,1.207384,-0.338296,1.608089,1.150430,-0.634676,-0.254125,-1.071040,1.211796,-0.278586,-0.525854,0.748559,0.164741,-0.481178,-1.675388,0.875207,0.975484,-0.257127,-0.012251,-2.161046,-0.796146,-0.836062,-0.694677,0.361975,-0.950141,-0.702509,-0.983462,0.284671,-1.131678,...,-1.792642,0.489132,0.707611,0.945068,0.635493,0.032225,-0.331405,1.627035,1.052389,-1.286518,-0.632650,-1.188038,-0.012820,0.857500,1.356536,-0.997797,0.729610,1.785720,-0.313547,-0.603171,1.687713,0.873078,0.883546,0.837203,-0.717129,-0.011016,0.617468,1.562592,0.885327,-1.959883,0.699144,0.089146,0.059228,0.732675,-1.229492,-1.434176,1.968918,0.379001,-0.353593,Nayara
4,-0.107481,-0.689445,-0.480524,0.131852,-1.843418,-0.476712,-0.859758,-1.044694,-1.539799,1.486555,-0.880955,1.304679,1.061523,0.134768,2.604447,0.981011,-0.818154,-0.131803,-0.571664,1.291709,-0.072231,-1.144796,1.688066,0.264710,0.972808,-2.386386,1.039902,1.534913,0.114218,0.220803,-0.803907,-1.023913,1.121686,0.105092,0.038132,-1.248504,1.253716,-1.648165,-0.489833,-0.070276,...,-1.092080,-0.621510,-1.314188,0.868450,0.994173,-0.501640,0.800490,0.194400,-0.756526,-1.231052,-0.405903,-0.023578,0.909067,0.289566,1.403794,-0.446976,-1.125021,0.379413,-0.716104,1.348528,0.722898,0.136608,-0.175436,1.756393,-1.363832,-0.106016,-0.615542,0.843601,0.634066,-0.125276,-0

In [ ]:
df.to_csv('/content/drive/My Drive/Colab Notebooks/faces.csv', index=False)